In [1]:
""" Plots the alpha band (8-12 Hz) power spectral density (PSD) of EEG/MEG data and empty room noise. """

import numpy as np
from pathlib import Path
import mne
import matplotlib.pyplot as plt
import json
from eeg_model import EEGData

script_path = Path().resolve()
folder = (script_path / "../").resolve()

JSON_PATH = script_path / "config.json"
EVENTS_PATH = script_path / "events.json"

with open(JSON_PATH, "r") as f:
    config = json.load(f)

with open(EVENTS_PATH, "r") as f:
    event_dict = json.load(f)

def	plot_clean_eeg(config, event_dict, l_freq, h_freq):

	# Get the sample data,
	sample_data_raw_file = folder / config["eeg_path"]
	sample_data_raw_noise = folder / config["noise_path"]

	eeg_data = EEGData(sample_data_raw_file, sample_data_raw_noise, l_freq=l_freq, h_freq=h_freq, verbose=True)	
	eeg_data.load_event_dict(event_dict)

	raw_data = eeg_data.get_raw_data()
	events, event_dict = eeg_data.get_events()

	eeg_data.filter_data(tmax=None, verbose=False)

	# Plot difference between EEG & Room Noise
	""" eeg_data.compute_psd()
	eeg_data.plot_psd() """

	# Based on the plot, choose the number of components that explain 95% of the variance
	pca = eeg_data.apply_pca()
	explained_variance = np.cumsum(pca.explained_variance_ratio_)
	n_components = np.argmax(explained_variance >= 0.99) + 1

	print(f"Number of components to explain 95% variance: {n_components}")

	# Apply PCA with the chosen number of components
	eeg_data.apply_pca(n_components=n_components)

	# Compute ICA
	clean_data = eeg_data.compute_ica(n_components, plot_comp=False,
    	plot_arts=False, verbose=False)
	# Plot cleaned EEG after ICA
	eeg_data.plot_clean_eeg()

	# Plot evoked events on EEG & MEG Data
	eeg_data.plot_evoked_events("Task 1 (open and close left or right fist)",
        "Task 2 (imagine opening and closing left or right fist)", type="eeg", verbose=False)
	eeg_data.plot_evoked_events("Task 1 (open and close left or right fist)",
        "Task 2 (imagine opening and closing left or right fist)", type="meg", verbose=False)

	eeg_data.plot_evoked_events("Task 3 (open and close both fists or both feet)",
        "Task 4 (imagine opening and closing both fists or both feet)", type="eeg", verbose=False)
	eeg_data.plot_evoked_events("Task 3 (open and close both fists or both feet)",
        "Task 4 (imagine opening and closing both fists or both feet)", type="meg", verbose=False)

plot_clean_eeg(config, event_dict, l_freq=0, h_freq=75)

Opening raw data file /Users/Shared/42/ML/projects/total_perspective_vortex/sample_data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
320 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]
Opening raw data file /Users/Shared/42/ML/projects/total_perspective_vortex/sample_data/MEG/sample/ernoise_raw.fif...
Isotrak not found
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 19800 ... 85867 =     32.966 ...   142.965 secs
Ready.
Reading 0 ... 66067  =      0.000 ...   109.999 secs...


ValueError: No matching events found for Task 4 (imagine opening and closing both fists or both feet) (event id 6)